Задание:

Сформировать три отдельных набора данных, представляющих собой временные ряды, многомерные данные и наборы текста из открытых источников. Приоритетным инструментом для сбора данных является использование API.


**Требования к данным/источникам:**

**Временные ряды.**

Источник данных: открытые API, предоставляющие данные временных рядов (например, финансовые данные, метеорологические данные, данные о заболевании).

Формат данных: даты и соответствующие им значения (например, цена акций, температура, количество случаев заболевания).

**Многомерные данные.**

Источник данных: открытые API, предоставляющие многомерные данные (например, данные о параметрах транспортных средств, данные о здоровье, спортивная статистика).

Формат данных: набор данных с несколькими измерениями (например, страна, год, показатель здоровья).

**Наборы текста.**
Источник данных: открытые API, предоставляющие текстовые данные (например, новости, статьи, социальные медиа).
Формат данных: текстовые записи с метаданными (например, дата публикации, автор, заголовок, текст).

**Этапы выполнения задания:**
1. Исследование и выбор API.
Найти подходящие API для каждого типа данных. Зарегистрироваться и получить ключи доступа к API, если это требуется.

2. Разработка скриптов для парсинга.
Написать скрипты на Python, использующие библиотеки requests или http.client для обращения к API. Обработать ответы от API и преобразовать их в удобный для анализа формат (например, pandas DataFrame для временных рядов и многомерных данных, список словарей для текстовых данных).


3. Сохранение данных.
Сохранить данные в подходящих форматах (например, CSV для временных рядов и многомерных данных, JSON или CSV для текстовых данных). Обеспечить сохранение метаданных (наличие ссылок на исходные данные).

### Временные ряды

In [1]:
import requests
import pandas as pd
import json
import time

In [3]:
# Конфигурация
API_KEY = '027cc5c4693246f5a2f2212e8e4b6a29'
SYMBOL = 'NFLX'  # Акции Netflix
BASE_URL = 'https://www.alphavantage.co/query'
PARAMS = {
    'function': 'TIME_SERIES_DAILY',
    'symbol': SYMBOL,
    'outputsize': 'full',  # Полная история (>20 лет)
    'apikey': API_KEY
}
OUTPUT_FILE = 'msft_stock_prices.csv'

# Запрос к API
try:
    response = requests.get(BASE_URL, params=PARAMS)
    response.raise_for_status()  # Проверка на HTTP-ошибки (e.g., 429 Too Many Requests)
    data = response.json()

    # Проверка на ошибки API
    if 'Error Message' in data:
        raise ValueError(f'API Error: {data['Error Message']}')
    if 'Note' in data:
        raise ValueError(f'API Limit Reached: {data['Note']}')
    if 'Time Series (Daily)' not in data:
        raise ValueError('No time series data in response')

    # Извлечение временных рядов
    time_series = data['Time Series (Daily)']
    records = [
        {
            'date': date,
            'open': float(values['1. open']),
            'high': float(values['2. high']),
            'low': float(values['3. low']),
            'close': float(values['4. close']),
            'volume': int(values['5. volume']),
            'source_url': f'{BASE_URL}?function=TIME_SERIES_DAILY&symbol={SYMBOL}&outputsize=full'
        }
        for date, values in time_series.items()
    ]

    # Создание DataFrame
    df = pd.DataFrame(records)
    df['date'] = pd.to_datetime(df['date'])  # Конвертация даты
    df = df.sort_values('date')  # Сортировка по дате

    # Сохранение в CSV
    df.to_csv(OUTPUT_FILE, index=False)
    print(f'Data saved to {OUTPUT_FILE}. Rows: {len(df)}')

except requests.exceptions.RequestException as e:
    print(f'HTTP Error: {e}')
except ValueError as e:
    print(f'Error: {e}')
except Exception as e:
    print(f'Unexpected Error: {e}')

HTTP Error: HTTPSConnectionPool(host='www.alphavantage.co', port=443): Max retries exceeded with url: /query?function=TIME_SERIES_DAILY&symbol=NFLX&outputsize=full&apikey=027cc5c4693246f5a2f2212e8e4b6a29 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7cd33ba54dd0>: Failed to resolve 'www.alphavantage.co' ([Errno -3] Temporary failure in name resolution)"))


### Многомерные данные

In [6]:
endpoint = f'https://api.chess.com/pub/country/RU/players'
num_players = 3000  
output_file = 'chess_ru_players.csv'
HEADERS = {'User-Agent': 'my-stats-tool/1.0 (contact: vika995511@gmail.com)'}

# список usernames
try:
    response = requests.get(endpoint, headers=HEADERS)
    response.raise_for_status()
    list_data = response.json()
    
    if 'players' not in list_data or not list_data['players']:
        raise ValueError('No players list found for Russia')
    
    usernames = list_data['players']
    print(f'Fetched {len(usernames)} usernames from {endpoint}')

except requests.exceptions.RequestException as e:
    print(f'Error fetching players list: {e}')
    exit(1)

# получение статистики по каждому игроку
usernames_to_process = usernames[:num_players]
records = []
for i, username in enumerate(usernames_to_process):
    stats_url = f'https://api.chess.com/player/{username}/stats'
    try:
        response = requests.get(stats_url, headers=HEADERS)
        response.raise_for_status()
        stats = response.json()
        
        bullet = stats.get('chess_bullet', {})
        blitz = stats.get('chess_blitz', {})
        tactics = stats.get('tactics', {})
        
        record = {
            'username': username,
            'country': 'RU',
            'bullet_last_rating': bullet.get('last', {}).get('rating', None),
            'bullet_best_rating': bullet.get('best', {}).get('rating', None),
            'bullet_wins': bullet.get('record', {}).get('win', None),
            'bullet_losses': bullet.get('record', {}).get('loss', None),
            'blitz_last_rating': blitz.get('last', {}).get('rating', None),
            'tactics_solved': tactics.get('solved', {}).get('total', {}).get('count', None),
            'source_url': stats_url
        }
        records.append(record)
        
        # Rate limit: 1/sec
        time.sleep(1)
        if (i + 1) % 100 == 0:
            print(f'Processed {i+1}/{len(usernames_to_process)}: {username}')
        
    except requests.exceptions.RequestException as e:
        print(f'HTTP Error for {username}: {e}')
        continue
    except Exception as e:
        print(f'Unexpected error for {username}: {e}')
        continue

# DataFrame и сохранение
if records:
    df = pd.DataFrame(records)
    # Numeric конвертация
    numeric_cols = ['bullet_last_rating', 'bullet_best_rating', 'bullet_wins', 'bullet_losses', 'blitz_last_rating', 'tactics_solved']
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    df = df.dropna(subset=['username'])
    
    df.to_csv(output_file, index=False)
    print(f'Data saved to {output_file}. Rows: {len(df)}')
    print(f'Numeric columns (>4 different): {numeric_cols}')
    print('Sample data:')
    print(df.head())
else:
    print('No data collected')


Error fetching players list: HTTPSConnectionPool(host='api.chess.com', port=443): Max retries exceeded with url: /pub/country/RU/players (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7a06443771d0>: Failed to resolve 'api.chess.com' ([Errno -3] Temporary failure in name resolution)"))


NameError: name 'usernames' is not defined

In [7]:
pip install retrying

ERROR: Could not find a version that satisfies the requirement retrying (from versions: none)
ERROR: No matching distribution found for retrying
Note: you may need to restart the kernel to use updated packages.


In [9]:
import requests
import pandas as pd
import time
import socket

In [10]:
# Конфигурация
BASE_URL = 'https://api.chess.com/pub'
COUNTRY_CODE = 'RU'
PRIMARY_ENDPOINT = f'{BASE_URL}/country/{COUNTRY_CODE}/players'
FALLBACK_ENDPOINT = f'{BASE_URL}/titled/GM'
NUM_PLAYERS = 1000  # Увеличь до 5000 для больше строк (время ~1.4 часа)
OUTPUT_FILE = 'chess_russia_players_stats.csv'
HEADERS = {'User-Agent': "my-stats-tool/1.0 (contact: your@email.com)"}  # Двойные кавычки для JSON

# Функция для повторов при DNS/HTTP ошибках
def get_with_retry(url, headers=None):
    max_attempts = 3
    for attempt in range(max_attempts):
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            return response.json()
        except (requests.exceptions.ConnectionError, socket.gaierror) as e:
            print(f'Attempt {attempt+1}/{max_attempts} failed for {url}: DNS/Connection error: {e}')
            if attempt + 1 == max_attempts:
                raise
            time.sleep(2)
        except requests.exceptions.HTTPError as e:
            print(f'Attempt {attempt+1}/{max_attempts} failed for {url}: HTTP error: {e}')
            if attempt + 1 == max_attempts:
                raise
            time.sleep(2)
        except requests.exceptions.RequestException as e:
            print(f'Attempt {attempt+1}/{max_attempts} failed for {url}: Request error: {e}')
            if attempt + 1 == max_attempts:
                raise
            time.sleep(2)
    raise ValueError('All retry attempts failed')

# Шаг 1: Получить список usernames
usernames = []
try:
    print(f'Trying primary endpoint: {PRIMARY_ENDPOINT}')
    list_data = get_with_retry(PRIMARY_ENDPOINT, HEADERS)
    
    if 'players' not in list_data or not list_data['players']:
        raise ValueError('No players list found')
    
    usernames = list_data['players']
    print(f'Fetched {len(usernames)} usernames from {PRIMARY_ENDPOINT}')

except Exception as e:
    print(f'Primary endpoint failed: {e}')
    print(f'Trying fallback endpoint: {FALLBACK_ENDPOINT}')
    try:
        list_data = get_with_retry(FALLBACK_ENDPOINT, HEADERS)
        if 'players' not in list_data or not list_data['players']:
            raise ValueError('No players list in fallback')
        usernames = list_data['players']
        print(f'Fetched {len(usernames)} usernames from {FALLBACK_ENDPOINT}')
    except Exception as e:
        print(f'Fallback endpoint failed: {e}')
        print('Exiting due to failure in both endpoints')
        exit(1)

# Шаг 2: Для первых NUM_PLAYERS получить stats
usernames_to_process = usernames[:NUM_PLAYERS]
records = []
for i, username in enumerate(usernames_to_process):
    stats_url = f'{BASE_URL}/player/{username}/stats'
    try:
        stats = get_with_retry(stats_url, HEADERS)
        
        bullet = stats.get('chess_bullet', {})
        blitz = stats.get('chess_blitz', {})
        tactics = stats.get('tactics', {})
        
        record = {
            'username': username,
            'country': COUNTRY_CODE if PRIMARY_ENDPOINT in stats_url else 'Various',
            'bullet_last_rating': bullet.get('last', {}).get('rating', None),
            'bullet_best_rating': bullet.get('best', {}).get('rating', None),
            'bullet_wins': bullet.get('record', {}).get('win', None),
            'bullet_losses': bullet.get('record', {}).get('loss', None),
            'blitz_last_rating': blitz.get('last', {}).get('rating', None),
            'tactics_solved': tactics.get('solved', {}).get('total', {}).get('count', None),
            'source_url': stats_url
        }
        records.append(record)
        
        time.sleep(1)  # Rate limit: 1/sec
        if (i + 1) % 100 == 0:
            print(f'Processed {i+1}/{len(usernames_to_process)}: {username}')
        
    except Exception as e:
        print(f'Error for {username}: {e}')
        continue

# Шаг 3: DataFrame и сохранение
if records:
    df = pd.DataFrame(records)
    numeric_cols = ['bullet_last_rating', 'bullet_best_rating', 'bullet_wins', 'bullet_losses', 'blitz_last_rating', 'tactics_solved']
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    df = df.dropna(subset=['username'])
    
    df.to_csv(OUTPUT_FILE, index=False)
    print(f'Data saved to {OUTPUT_FILE}. Rows: {len(df)}')
    print(f'Numeric columns (>4 different): {numeric_cols}')
    print('Sample data:')
    print(df.head())
else:
    print('No data collected')

Trying primary endpoint: https://api.chess.com/pub/country/RU/players
Attempt 1/3 failed for https://api.chess.com/pub/country/RU/players: DNS/Connection error: HTTPSConnectionPool(host='api.chess.com', port=443): Max retries exceeded with url: /pub/country/RU/players (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7bb459256310>: Failed to resolve 'api.chess.com' ([Errno -3] Temporary failure in name resolution)"))
Attempt 2/3 failed for https://api.chess.com/pub/country/RU/players: DNS/Connection error: HTTPSConnectionPool(host='api.chess.com', port=443): Max retries exceeded with url: /pub/country/RU/players (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7bb4592549d0>: Failed to resolve 'api.chess.com' ([Errno -3] Temporary failure in name resolution)"))
Attempt 3/3 failed for https://api.chess.com/pub/country/RU/players: DNS/Connection error: HTTPSConnectionPool(host='api.chess.com', port=443): Max retries exceeded wit

### Наборы текста